# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846899784222                   -0.70    4.5         
  2   -7.852328190332       -2.27       -1.53    1.0   24.9ms
  3   -7.852616627594       -3.54       -2.56    1.5   25.8ms
  4   -7.852645986093       -4.53       -2.91    2.5   32.7ms
  5   -7.852646529422       -6.26       -3.24    1.0   23.9ms
  6   -7.852646679432       -6.82       -4.10    1.2   23.8ms
  7   -7.852646686685       -8.14       -5.23    2.2   31.3ms
  8   -7.852646686726      -10.38       -5.50    1.8   28.7ms
  9   -7.852646686729      -11.59       -5.73    1.0   24.9ms
 10   -7.852646686730      -11.99       -6.81    1.2   25.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846813461778                   -0.70           4.5         
  2   -7.852525027301       -2.24       -1.64   0.80    2.2    253ms
  3   -7.852635196621       -3.96       -2.73   0.80    1.0   22.4ms
  4   -7.852646492743       -4.95       -3.22   0.80    2.0   28.7ms
  5   -7.852646672299       -6.75       -4.04   0.80    1.2   22.8ms
  6   -7.852646686285       -7.85       -4.77   0.80    1.8   26.5ms
  7   -7.852646686723       -9.36       -5.62   0.80    1.8   27.6ms
  8   -7.852646686730      -11.16       -6.94   0.80    2.0   26.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.417143e+01     3.125283e+00
 * time: 0.5153770446777344
     1     1.307662e+00     1.577557e+00
 * time: 0.7091920375823975
     2    -1.703971e+00     1.976171e+00
 * time: 0.7323319911956787
     3    -3.766727e+00     1.575089e+00
 * time: 0.7657439708709717
     4    -4.926103e+00     1.594999e+00
 * time: 0.7997479438781738
     5    -6.694624e+00     1.069441e+00
 * time: 0.8323988914489746
     6    -7.426489e+00     5.894119e-01
 * time: 0.8659858703613281
     7    -7.666770e+00     5.252888e-01
 * time: 0.8887619972229004
     8    -7.784746e+00     2.401675e-01
 * time: 0.9115920066833496
     9    -7.818638e+00     7.933464e-02
 * time: 0.9342739582061768
    10    -7.835902e+00     5.600867e-02
 * time: 0.9570648670196533
    11    -7.842771e+00     4.909402e-02
 * time: 0.9797699451446533
    12    -7.844470e+00     5.182474e-02
 * time: 1.002619981765747
    13    -7.845263e+00     4.866107e-02
 * time: 1.03515791893

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846881161369                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645919204                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.13s
  3   -7.852646686730      -13.34       -7.25    142ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.737193788652581e-7
|ρ_newton - ρ_scfv| = 2.214969988342282e-7
|ρ_newton - ρ_dm|   = 7.891172347294021e-10
